In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

#Initialize SparkSession
spark = SparkSession.builder.appName("WordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 03:52:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext

In [4]:
test_df = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]


In [5]:
ud_scehma = ["employee_name","department","state","salary","age","bonus"]

In [6]:
df = spark.createDataFrame(data=test_df,schema = ud_scehma)

In [7]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [16]:
from pyspark.sql.functions import *
df.cache().count()

23/09/25 04:38:16 WARN CacheManager: Asked to cache already cached data.


9

In [20]:
from pyspark import StorageLevel
test = StorageLevel(useDisk=False,useMemory=True,useOffHeap=False,deserialized=False,replication=2)
df.persist(storageLevel=test)

23/09/25 04:42:23 WARN CacheManager: Asked to cache already cached data.


DataFrame[employee_name: string, department: string, state: string, salary: bigint, age: bigint, bonus: bigint]

In [14]:
import sys

In [15]:
print(sys.version)

3.11.4 (main, Jul  5 2023, 14:15:25) [GCC 11.2.0]


In [21]:
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [27]:
df.groupBy("department","state").agg(sum("salary").alias("sum_salary"), \
                             avg("salary").alias("avg_salary")) \
                            .where(col("avg_salary")>80000).show()

+----------+-----+----------+----------+
|department|state|sum_salary|avg_salary|
+----------+-----+----------+----------+
|     Sales|   CA|     81000|   81000.0|
|   Finance|   CA|    189000|   94500.0|
|     Sales|   NY|    176000|   88000.0|
|   Finance|   NY|    162000|   81000.0|
| Marketing|   NY|     91000|   91000.0|
+----------+-----+----------+----------+



In [26]:
df.createOrReplaceTempView("ida")

In [28]:
test = spark.sql("select * from ida")

In [29]:
test.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [33]:
spark.sql("CREATE DATABASE idashell")
spark.sql("USE idashell")

df.write.saveAsTable("emp_tbl")

In [34]:
test = spark.sql("DESCRIBE emp_tbl")
test.show()

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|employee_name|   string|   null|
|   department|   string|   null|
|        state|   string|   null|
|       salary|   bigint|   null|
|          age|   bigint|   null|
|        bonus|   bigint|   null|
+-------------+---------+-------+



In [35]:
test01 = spark.sql("DESCRIBE EXTENDED emp_tbl")
test01.show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|       employee_name|              string|   null|
|          department|              string|   null|
|               state|              string|   null|
|              salary|              bigint|   null|
|                 age|              bigint|   null|
|               bonus|              bigint|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|            idashell|       |
|               Table|             emp_tbl|       |
|        Created Time|Mon Sep 25 05:24:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|         Spark 3.4.1|       |
|                Type|             MANAGED|       |
|            Provider|             parquet|       |
|           

In [39]:
spark.sql("CREATE DATABASE idashell2")
spark.sql("USE idashell2")
df.write.option("path","/home/labuser/Documents/ida_test/").saveAsTable("emp01_tbl")

In [40]:
spark.sql("drop table idashell.emp_tbl")

DataFrame[]

In [41]:
spark.sql("drop table idashell2.emp01_tbl")

DataFrame[]

In [42]:
df.write.partitionBy("state","department").csv("/home/labuser/Documents/test01")

In [43]:
def f(df):
    print(df.state)

In [44]:
test=df.foreach(f)

CA
NY
NY
CA
NY
NY
NY
CA
CA


In [45]:
json_data = [
    '{"name": "Alice", "age": 25}',
    '{"name": "Bob", "age": 30}',
    '{"name": "Charlie", "age": 35}'
]

from pyspark.sql.functions import *
from pyspark.sql.types import *
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True)
])

df = spark.read.schema(schema).json(spark.sparkContext.parallelize(json_data))
df.show()

+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [46]:
json_data = [
    '{"name": "Alice", "age": 25, "address": {"city": "New York", "state": "NY"}}',
    '{"name": "Bob", "age": 30, "address": {"city": "San Francisco", "state": "CA"}}',
    '{"name": "Charlie", "age": 35, "address": {"city": "Los Angeles", "state": "CA"}}'
]

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("address", StructType([
        StructField("city", StringType(), True),
        StructField("state", StringType(), True)
    ]), True)
])


df = spark.read.schema(schema).json(spark.sparkContext.parallelize(json_data))
df.show()

+-------+---+-------------------+
|   name|age|            address|
+-------+---+-------------------+
|  Alice| 25|     {New York, NY}|
|    Bob| 30|{San Francisco, CA}|
|Charlie| 35|  {Los Angeles, CA}|
+-------+---+-------------------+

